In [ ]:
def sin(x, T=100):
    return np.sin(2.0 * np.pi * x /T)

def toy_problem(T=100, ampl=0.05):
    x = np.arange(0, 2*T+1)
    noise = ampl * np.random.uniform(low=-1.0, high = 1.0, size=len(x))
    return sin(x) +noise

In [ ]:
T=100
f = toy_problem(T)

In [ ]:
length_of_sequences = 2*T
maxlen = 25

data = []
target = []

for i in range(0, lenght_of_sequences - maxlen +1):
    data.append(f[i: i + maxlen])
    target.append(f[i + maxlen])

In [ ]:
X = np.array(data).reshape(len(data), maxlen, 1)

In [ ]:
Y = np.array(target).reshape(len(data), 1)

In [ ]:
X = np.zeros((len(data), maxlen, 1), dtype=float)
Y = np.zeros((len(data), 1), dtype=float)

for i, seq in enumerate(data):
    for t, value in enumerate(seq):
        X[i, t, 0] =value
    Y[i,0] = target[i]

In [ ]:
N_train = int(len(data)* 0.9)
N_validation = len(data) - N_train

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=N_validation)

In [ ]:
# from keras.layers.recurrent import simpleRNN
from keras.layers.recurrent import LSTM

In [ ]:
model = Sequential()
# model.add(SimpleRNN(n_hidden, 
#                     init = weight_variable, 
#                     input_shape = (maxlen, n_out)))
model.add(LSTM(n_hidden, 
                    init = weight_variable, 
                    input_shape = (maxlen, n_in)))
model.add(Dense(n_out, init=weight_variable))
model.add(Activation('linear'))

In [ ]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2 = 0.999)
model.compile(loss='mean_squared_error', optimizer = optimizer)

In [ ]:
epochs=500
batch_size=10

history= model.fit(X_train, Y_train, batch_size = batch_size,
                  epochs=epochs, validation_daa = (X_validation, Y_validation),
                  callbacks=[early_stopping])

In [ ]:
truncate = maxlen
Z = X[:1]

original = [f[i] for i in range(maxlen)]
predicted = [None for i in range(maxlen)]

for i in range(length_of_sequences - maxlen +1):
    z_ = Z[-1:]
    y_ = model.predict(z_)
    sequence_ = np.concatemate(
        (z_.reshape(maxlen, n_in)[1:], y_),
        axis=0).reshape(1, maxlen, n_in)
    Z = np.append(Z, sequence_, axis=0)
    predicted.append(y_.reshape(-1))